# Section 3: crewAI를 활용한 Multi-Agent 기술 분석 시스템 구축

하나의 '만능 Agent'에게 복잡한 업무를 맡기는 것은 LLM에게 인지 과부하(Cognitive Overload)를 유발하여 성능 저하의 원인이 됩니다.

이번 시간에는 각자 명확한 전문성을 가진 여러 AI Agent들이 하나의 '팀(Crew)'을 이루어 협력하며, 복합적인 과업을 자율적으로 수행하는 **Multi-Agent 시스템**을 `crewAI` 프레임워크를 통해 구축해 보겠습니다. 이를 통해 단일 Agent의 한계를 극복하고, 보다 정교하고 신뢰성 높은 자동화 시스템을 설계하는 방법을 학습합니다.

### 3.1. crewAI 설치 및 API 키 설정

Multi-Agent 시스템 구축을 위해 `crewAI`와 관련 도구 라이브러리들을 설치합니다. 이번 실습에서는 두 종류의 외부 API 키가 필요합니다.

- **Google API Key:** Agent들의 두뇌 역할을 하는 LLM(`gemini-2.5-pro`)을 사용하기 위함입니다.
- **Serper API Key:** '리서처 Agent'가 최신 정보를 얻기 위해 실제 인터넷 검색을 수행하기 위함입니다.

API 키는 Colab의 보안 비밀 기능을 사용하여 안전하게 관리합니다.

In [1]:
# crewai와 관련 도구들을 설치합니다.
# !pip install crewai=="0.150.0" crewai_tools langchain_google_genai -q

#### API 키 발급 및 설정 안내 (모두 무료)

1.  **Google API Key:** Lv4 1일차 첫 실습에서 이미 설정했습니다. `GOOGLE_API_KEY`라는 이름으로 Colab Secrets에 저장되어 있는지 확인합니다.
2.  **Serper API Key:**
    - `serper.dev` 사이트에 접속하여 가입합니다.
    - 로그인 후 대시보드에서 API Key를 확인하고 복사합니다. (무료 플랜으로 실습에 충분한 쿼리가 제공됩니다.)
    - Colab Secrets에 `SERPER_API_KEY` 라는 이름으로 저장하고, 노트북 액세스를 활성화합니다.

**※ 참고: crewAI와 LiteLLM의 API Key 환경변수**
`crewAI`는 내부적으로 `LiteLLM`을 사용하여 다양한 LLM을 지원합니다. `LiteLLM`은 각 LLM 제공사별로 다른 환경변수 이름을 사용하는데, Gemini의 경우 `GEMINI_API_KEY`를 기본값으로 인식합니다. 따라서 `google-generativeai` 라이브러리가 사용하는 `GOOGLE_API_KEY`와 호환성을 위해 두 이름으로 모두 키를 설정해주는 것이 안정적입니다. ([LiteLLM 공식 문서](https://docs.litellm.ai/docs/providers)에서 다른 LLM들의 환경변수 이름을 확인할 수 있습니다.)

In [5]:
import os
from getpass import getpass

# API 키 설정
if "GOOGLE_API_KEY" not in os.environ:
    api_key = getpass("Google AI Studio API 키를 입력하세요: ")
    os.environ["GOOGLE_API_KEY"] = api_key
    # crewAI의 LiteLLM 호환성을 위한 설정
    os.environ["GEMINI_API_KEY"] = api_key

# Serper API Key (웹 검색 Tool용)
if "SERPER_API_KEY" not in os.environ:
    os.environ["SERPER_API_KEY"] = getpass("SERPER_API_KEY 를 입력하세요: ")

print("✅ 모든 API 키가 성공적으로 설정되었습니다.")

✅ 모든 API 키가 성공적으로 설정되었습니다.


### 3.2. Custom Tool 정의 (수학/공학적 분석)

LLM은 복잡하고 정밀한 수학 계산에 약점을 보입니다. 이는 LLM이 확률 기반의 언어 모델이기 때문입니다. 이러한 약점을 보완하기 위해, 안정적인 Python 코드로 계산 로직을 구현하고 이를 Agent가 사용할 수 있는 **맞춤형 Tool(Custom Tool)**로 제공해야 합니다. 이는 Agent에게 LLM이 본질적으로 가지지 못하는 '계산 능력'이라는 슈퍼파워를 부여하는 것과 같습니다.

`crewAI`는 `@tool` 데코레이터를 통해 Python 함수를 매우 쉽게 Agent의 Tool로 변환할 수 있도록 지원합니다. 여기서는 GPU의 성능(벤치마크 점수)과 전력 소비량(TGP)을 입력받아, '와트당 성능(전성비)'이라는 중요한 공학적 지표를 계산하는 Tool을 만들어 보겠습니다.

In [6]:
from crewai.tools import tool


@tool("Performance Per Watt Calculator")
def performance_calculator_tool(
    product_a_performance: float, product_a_power: float, product_b_performance: float, product_b_power: float
) -> str:
    """두 제품의 성능 점수와 전력 소비량을 입력받아, 와트당 성능(전성비)을 계산하고 비교하는 데 사용됩니다."""
    try:
        # 전력 소비량이 0 이하인 경우 계산이 불가능하므로 오류를 방지합니다.
        if product_a_power <= 0 or product_b_power <= 0:
            return "오류: 전력 소비량(power)은 0보다 커야 합니다."

        perf_per_watt_a = product_a_performance / product_a_power
        perf_per_watt_b = product_b_performance / product_b_power

        if perf_per_watt_a > perf_per_watt_b:
            comparison = f"제품 A가 제품 B보다 전성비가 약 {perf_per_watt_a / perf_per_watt_b:.2f}배 더 좋습니다."
        elif perf_per_watt_b > perf_per_watt_a:
            comparison = f"제품 B가 제품 A보다 전성비가 약 {perf_per_watt_b / perf_per_watt_a:.2f}배 더 좋습니다."
        else:
            comparison = "두 제품의 전성비가 동일합니다."

        return (
            f"제품 A (성능:{product_a_performance}, 전력:{product_a_power}W)의 전성비: {perf_per_watt_a:.2f} 점/W\n"
            f"제품 B (성능:{product_b_performance}, 전력:{product_b_power}W)의 전성비: {perf_per_watt_b:.2f} 점/W\n"
            f"비교 분석: {comparison}"
        )
    except Exception as e:
        return f"계산 중 예상치 못한 오류 발생: {e}"


print("✅ Custom Tool (performance_calculator_tool)이 성공적으로 정의되었습니다.")

✅ Custom Tool (performance_calculator_tool)이 성공적으로 정의되었습니다.


### 3.3. Agent (팀원) 정의

이제 우리 Crew를 구성할 세 명의 전문가 Agent를 정의합니다. 각 Agent는 명확한 **역할(role), 목표(goal), 배경 설정(backstory)**을 가집니다. 이는 단순한 설명이 아니라, LLM에게 강력한 **'페르소나(Persona)'**를 부여하는 정교한 프롬프트 엔지니어링 기법입니다. 이를 통해 각 Agent는 자신의 역할에 맞는 톤앤매너와 전문성을 가지고 추론하고 행동하게 됩니다.

- **선임 기술 리서처:** 최신 정보를 웹에서 검색하고 사실 관계를 확인하는 역할을 합니다.
- **데이터 분석가:** 수치 데이터를 바탕으로 공학적 계산을 수행하고 객관적인 분석을 제공합니다.
- **기술 보고서 작성 전문가:** 정성적 정보와 정량적 데이터를 종합하여 최종 보고서를 작성합니다.

In [7]:
from crewai import Agent, LLM
from crewai_tools import SerperDevTool

# Tool 초기화
search_tool = SerperDevTool()

# LLM을 Gemini 2.5 Pro로 설정합니다.
llm = LLM(model="gemini/gemini-2.5-pro", temperature=0.1)

# Agent 정의
researcher = Agent(
    role="선임 기술 리서처 (Senior Tech Researcher)",
    goal="NVIDIA와 AMD의 최신 GPU에 대한 스펙, 성능 벤치마크, 가격 등 신뢰할 수 있는 정보를 웹에서 검색하여 수집하고, 이를 구조화된 데이터로 정리합니다.",
    backstory="당신은 20년 경력의 반도체 전문 저널리스트로, 루머를 걸러내고 검증된 출처의 데이터만을 찾아내는 능력이 탁월합니다. 당신의 보고는 항상 정확한 수치와 출처를 기반으로 합니다.",
    tools=[search_tool],
    llm=llm,
    allow_delegation=False,  # 이 Agent는 다른 Agent에게 작업을 위임하지 않습니다.
    verbose=True,
)

analyst = Agent(
    role="데이터 분석가 (Data Analyst)",
    goal="리서처가 수집한 구조화된 데이터를 바탕으로 각 GPU의 '와트당 성능'과 같은 핵심 지표를 계산하고, 객관적인 수치를 기반으로 성능을 비교 분석합니다.",
    backstory="당신은 MIT에서 컴퓨터 공학을 전공한 데이터 과학자입니다. 숫자를 통해 제품의 진정한 가치를 꿰뚫어 보는 것을 즐기며, 모든 분석은 데이터에 근거하여 명확하고 논리적으로 제시합니다.",
    tools=[performance_calculator_tool],
    llm=llm,
    allow_delegation=False,
    verbose=True,
)

writer = Agent(
    role="기술 보고서 작성 전문가 (Tech Report Writer)",
    goal="리서처의 정성적 정보와 분석가의 정량적 데이터를 종합하여, 최종적으로 경영진이 의사결정에 활용할 수 있도록 명확하고 간결한 기술 비교 보고서를 작성합니다.",
    backstory="당신은 기술 컨설팅 펌의 수석 컨설턴트로, 복잡한 기술 내용을 비전문가도 쉽게 이해할 수 있는 비즈니스 언어로 바꾸는 데 매우 능숙합니다. 보고서의 구조와 논리적 흐름을 가장 중요하게 생각합니다.",
    llm=llm,
    allow_delegation=False,
    verbose=True,
)

### 3.4. Task (업무) 정의 및 안정적인 데이터 구조 설계

각 Agent에게 부여할 구체적인 **업무 지시서(Task)**를 작성합니다. 여기서 가장 중요한 것은 Agent 간의 **데이터 전달 방식**입니다.

불안정한 자연어 텍스트 대신, `Pydantic` 모델을 사용하여 Agent 간에 주고받을 데이터의 구조를 명확하게 정의합니다. `task_research`에 `output_pydantic`을 설정하여 '리서처'가 반드시 이 구조에 맞춰 결과를 출력하도록 강제합니다. 이를 통해 '분석가'는 신뢰할 수 있는 데이터를 입력받아 오류 없이 다음 단계를 수행할 수 있게 됩니다.

In [8]:
from crewai import Task
from pydantic import BaseModel, Field


# Agent 간 데이터 전달을 위한 Pydantic 모델 정의
class GPUInfo(BaseModel):
    model_name: str = Field(description="GPU 모델명")
    timespy_score: int = Field(description="3DMark Time Spy 벤치마크 그래픽 점수")
    tgp_watts: int = Field(description="TGP 또는 TBP 값 (단위: Watts)")
    msrp_usd: int = Field(description="출시 당시 MSRP (단위: USD)")


# 리서치 Task: Pydantic 모델로 구조화된 출력을 강제합니다.
task_research = Task(
    description=(
        "NVIDIA GeForce RTX 4090과 AMD Radeon RX 7900 XTX GPU의 다음 정보를 찾아 정확한 수치와 함께 정리하세요:\n"
        "- 3DMark Time Spy 벤치마크 점수 (Graphics Score)\n"
        "- TGP (Total Graphics Power) 또는 TBP (Total Board Power) 값 (단위: Watts)\n"
        "- 출시 당시의 MSRP (권장소비자가격) (단위: USD)"
    ),
    expected_output=("각 GPU 모델별 3DMark Time Spy 점수, TGP(W), MSRP(USD)가 포함된 2개의 Pydantic 객체 리스트."),
    agent=researcher,
    output_pydantic=GPUInfo,
)

# 분석 Task: 이전 Task의 구조화된 결과를 context로 받아 작업을 수행합니다.
task_analyze = Task(
    description=(
        "앞선 리서치 단계에서 전달받은 두 GPU의 구조화된 데이터를 사용하여, 'Performance Per Watt Calculator' Tool을 호출하세요. "
        "두 GPU의 와트당 성능(전성비)을 계산하고, 그 결과를 바탕으로 가격 대비 성능에 대한 분석을 추가하세요."
    ),
    expected_output=("계산된 와트당 성능 수치와, 두 제품의 가격과 전성비를 종합적으로 고려한 명확한 비교 분석 결과."),
    agent=analyst,
    context=[task_research],
)

# 보고서 작성 Task: 이전 두 Task의 결과를 모두 종합하여 최종 보고서를 작성합니다.
task_write = Task(
    description=(
        "지금까지의 리서치 및 분석 결과를 모두 종합하여, 최종적으로 'NVIDIA 4090 vs AMD 7900XTX 성능 비교' 보고서를 작성하세요."
        "보고서는 다음 구조를 따라야 합니다:\n"
        "1. **개요:** 두 제품에 대한 간략한 소개.\n"
        "2. **핵심 성능 비교:** 3DMark Time Spy 점수를 기반으로 한 순수 성능 비교.\n"
        "3. **전력 효율성 분석:** 계산된 와트당 성능(전성비)을 기반으로 한 효율성 비교.\n"
        "4. **가격 및 시장 포지셔닝:** 출시 가격을 바탕으로 한 각 제품의 타겟 시장 분석.\n"
        "5. **최종 결론:** 어떤 사용자에게 어떤 제품을 추천할 수 있는지에 대한 종합적인 결론."
    ),
    expected_output=("위 구조에 맞춰 작성된, 경영진 보고용으로도 손색없는 완성도 높은 최종 기술 비교 보고서."),
    agent=writer,
    context=[task_research, task_analyze],
)

### 3.5. Crew (팀) 생성 및 실행
정의된 Agent와 Task들을 `Crew`로 묶어 하나의 팀을 구성합니다. 업무 처리 방식은 `Process.sequential`로 설정하여, Task가 정의된 순서대로 실행되도록 합니다.

`verbose=True`로 설정하면 각 Agent가 어떤 **생각(Thought)**을 하고 어떤 **행동(Action)**을 하는지 모든 과정을 상세하게 관찰할 수 있습니다. 이를 통해 Multi-Agent 시스템의 내부 동작 원리를 깊이 있게 이해할 수 있습니다.

In [ ]:
from crewai import Crew, Process
from IPython.display import display, Markdown

# Crew를 구성합니다.
gpu_analysis_crew = Crew(
    agents=[researcher, analyst, writer],
    tasks=[task_research, task_analyze, task_write],
    process=Process.sequential,  # 순차적 프로세스로 실행합니다.
    verbose=True,
)

# Crew 실행을 시작합니다.
# (실제 웹 검색과 LLM 호출이 여러 번 발생하므로, 실행에 1~2분 정도 소요될 수 있습니다.)
result = gpu_analysis_crew.kickoff()

print("\n\n" + "=" * 80)
print("                           최종 보고서")
print("=" * 80)
display(Markdown(result))

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 0878931f-5954-4de6-b0ce-e1b9866fe432                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: 선임 기술 리서처 (Senior Tech Researcher)                                                               │
│                                                                                                                 │
│  Task: NVIDIA GeForce RTX 4090과 AMD Radeon RX 7900 XTX GPU의 다음 정보를 찾아 정확한 수치와 함께 정리하세요:   │
│  - 3DMark Time Spy 벤치마크 점수 (Graphics Score)                                                               │
│  - TGP (Total Graphics Power) 또는 TBP (Total Board Power) 값 (단위: Watts)                                     │
│  - 출시 당시의 MSRP (권장소비자가격) (단위: USD)                                                                │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()